In [2]:
import numpy as np
from scipy.spatial import distance_matrix
from gurobipy import *
from scipy.spatial import ConvexHull
#from shapely.geometry import Polygon, Point
from numpy import random
import time
import pandas as pd

In [3]:
# 행정동 나중에 바꾸기
입지후보지 = pd.read_csv('입지선정후보군_좌표추가.csv', encoding='EUC-KR')
입지후보지 = 입지후보지[입지후보지['행정동'] == '석관동']

버스 = pd.read_csv('서울시_버스정류장_행정동추가.csv')
버스 = 버스[버스['행정동'].isin(['석관동','월곡2동','회기동','이문1동','이문2동','묵2동','공릉1동',\
                                 '월계3동','월계1동','장위1동','장위2동','장위3동'])]

지하철 = pd.read_csv('서울시_지하철역_행정동추가.csv', encoding='EUC-KR', index_col=0)
지하철 = 지하철[지하철['행정동'].isin(['석관동','월곡2동','회기동','이문1동','이문2동','묵2동','공릉1동',\
                                       '월계3동','월계1동','장위1동','장위2동','장위3동'])]

주차장 = pd.read_csv('서울시_주차장_행정동추가.csv', encoding='EUC-KR', index_col=0).drop('Unnamed: 0.1', axis=1)
주차장 = 주차장[주차장['행정동'].isin(['석관동','월곡2동','회기동','이문1동','이문2동','묵2동','공릉1동',\
                                       '월계3동','월계1동','장위1동','장위2동','장위3동'])]

주택 = pd.read_csv('서울시주택_행정동추가.csv', encoding='EUC-KR', index_col=0).drop('Unnamed: 0.1', axis=1)
주택 = 주택[주택['행정동'].isin(['석관동','월곡2동','회기동','이문1동','이문2동','묵2동','공릉1동',\
                                 '월계3동','월계1동','장위1동','장위2동','장위3동'])]

In [13]:
# # 버스, 지하철, 주차장, 주택 한 번에 넣을 떄
# X = list(버스['X좌표']) + list(지하철['경도']) + list(주차장['경도']) + list(주택['위도'])     # 주택 위도 경도 이름 바뀜
# Y = list(버스['Y좌표']) + list(지하철['위도']) + list(주차장['위도']) + list(주택['경도'])
# points = np.array([list(i) for i in zip(X, Y)])

In [14]:
# 버스, 지하철, 주차장, 주택 나눠서 정의
X = list(버스['X좌표']); Y = list(버스['Y좌표'])
points1 = np.array([list(i) for i in zip(X, Y)])

X = list(지하철['경도']); Y = list(지하철['위도'])
points2 = np.array([list(i) for i in zip(X, Y)])

X = list(주차장['경도']); Y = list(주차장['위도'])
points3 = np.array([list(i) for i in zip(X, Y)])

X = list(주택['위도']); Y = list(주택['경도'])
points4 = np.array([list(i) for i in zip(X, Y)])

In [ ]:
# 수요지점 = pd.DataFrame(points, columns=['경도','위도'])
# 수요지점.to_csv('석관동_수요지점.csv', index=False)

In [ ]:
# 문제 -> setObjective()의 수정 
def mclp(points,K,radius):
    """
    Solve maximum covering location problem
    Input:
        points: 버스정류장, 지하철역 위치 좌표 등 (기타 인근에 있으면 좋은 시설 좌표)
        K: 배치할 노인놀이터의 수
        radius: 반경 (노인들이 이동하기 적합한 거리)
        M: generate_candidate_sites 함수에서 생성할 random 좌표 수 (임의의 노인놀이터 수)
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)

    start = time.time()
    sites = np.array([list(i) for i in zip(입지후보지['x좌표'], 입지후보지['y좌표'])])
    J = sites.shape[0]                                         # 임의의 노인놀이터 좌표 개수
    
    I = points.shape[0]                                        # 버스정류장, 지하철역 위치 좌표 개수
    
    
    D = distance_matrix(points,sites)                          # 노인놀이터와 버스/지하철역 좌표 간 거리 계산
    mask1 = D<=radius
    D[mask1]=1                                                 # 버스/지하철역 좌표별로 노인놀이터의 반경 내 속하면 1, 아니면 0
    D[~mask1]=0
    # Build model
    m = Model()
    # Add variables
    x = {}
    y = {}
    for i in range(I):                                        # 버스/지하철역 좌표 개수만큼 반복
        y[i] = m.addVar(vtype=GRB.BINARY, name="y%d" % i)     # 모델에 결정 변수 추가 (변수 유형은 BINARY. 위치별 인수, 변수 이름 y+숫자)
    for j in range(J):
        x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)     # 노인놀이터 좌표 개수만큼 반복하여 변수 추가

    m.update()
    # Add constraints
    m.addConstr(quicksum(x[j] for j in range(J)) == K)

    for i in range(I):
        m.addConstr(quicksum(x[j] for j in np.where(D[i]==1)[0]) >= y[i])

    m.setObjective(quicksum(y[i] for i in range(I)),GRB.MAXIMIZE)
    m.setParam('OutputFlag', 0)
    m.optimize()
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objVal)
    
    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            # print v.varName,v.x
            if v.x==1 and v.varName[0]=="x":
                solution.append(int(v.varName[1:]))
    opt_sites = sites[solution]
    return opt_sites,m.objVal